<summary>Table of Contents</summary>

- [1. Connecting to CUDA](#1-connecting-to-cuda)
- [2. Informer](#2-informer)
- [3. PatchTST 336](#3-patchtst-336)
- [4. PatchTST 512](#4-patchtst-512)


Script with Informer and PatchTST (default parameters). PatchTST with seq_len = 336 and 512.

In [1]:
# Import libraries
import torch
import pandas as pd
import subprocess
import os
import time
import shutil

from utils.helper import extract_metrics_from_output, running_time, convert_results_into_df

# 1. Connecting to CUDA

Run on GPU, because running it on CPU will cost a lot of time.


I do not recommend to run it in Google Colab, because it interrupts training process.

If you are not going to use remote servers with multiple GPUs, skip this part.

In [2]:
# For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [3]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [4]:
torch.cuda.get_device_name(1)

'Tesla V100-PCIE-32GB'

In [5]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
cuda_device = "1"

# 2. Informer

In [6]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/informer/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [7]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]
seq_len = 96
model = "Informer"
itr = 2

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.0001
#n_heads = 16
e_layers = 2
d_layers = 1
loss = "MAE"

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results = []

In [29]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --d_layers {d_layers} \
              --factor 5 \
              --enc_in {num_cols[i]} \
              --dec_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --loss_fnc "{loss}" \
              --scaler_type minmax \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                informer_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=5, learning_rate=0.000

In [30]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/informer'

if not os.path.exists(path):
    os.makedirs(path)

informer_df = convert_results_into_df(informer_results, if_loss_fnc=False)

# Final DF
informer_df.columns = pd.MultiIndex.from_product([['Informer'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
informer_df.to_csv(os.path.join(path, 'informer.csv'))
informer_df.round(4)

Model            Informer                
Metrics               MSE    RMSE     MAE
Country Pred_len                         
DE      24         0.0257  0.1603  0.1028
        96         0.0427  0.2066  0.1433
        168        0.0503  0.2243  0.1545
ES      24         0.0267  0.1632  0.0977
        96         0.0608  0.2465  0.1540
        168        0.0550  0.2343  0.1495
FR      24         0.0130  0.1139  0.0647
        96         0.0222  0.1491  0.0907
        168        0.0239  0.1547  0.0971
GB      24         0.0366  0.1903  0.1242
        96         0.0557  0.2361  0.1630
        168        0.0570  0.2386  0.1661
IT      24         0.0122  0.1102  0.0646
        96         0.0211  0.1451  0.0898
        168        0.0220  0.1482  0.0937

# 3. PatchTST 336

We separated PatchTST from Informer, because it has additional arguments. It is not so easy to modify f-string (as e. g. distionary) to unpack some arguments with if statement. Moreover, it has different parameter values.

In [12]:
log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 336
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128 

# List to store the results
patchtst_results = []

In [13]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [14]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'

if not os.path.exists(path):
    os.makedirs(path)

patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['PatchTST/42'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_128.csv'))
patchtst_df.round(4)

Model            PatchTST/42                
Metrics                  MSE    RMSE     MAE
Country Pred_len                            
DE      24            0.0210  0.1448  0.0877
        96            0.0377  0.1941  0.1283
        168           0.0393  0.1981  0.1339
ES      24            0.0098  0.0988  0.0592
        96            0.0185  0.1361  0.0861
        168           0.0207  0.1439  0.0916
FR      24            0.0100  0.1000  0.0548
        96            0.0193  0.1390  0.0808
        168           0.0213  0.1460  0.0868
GB      24            0.0250  0.1582  0.0996
        96            0.0418  0.2045  0.1395
        168           0.0447  0.2115  0.1468
IT      24            0.0101  0.1004  0.0567
        96            0.0188  0.1371  0.0807
        168           0.0210  0.1448  0.0872

# 4. PatchTST 512

In [8]:
log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 512
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_512.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128 

# List to store the results
patchtst_results = []

In [9]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=512, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [10]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'

if not os.path.exists(path):
    os.makedirs(path)

patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['PatchTST/64'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_bs128_pl512.csv'))
patchtst_df.round(4)

Model            PatchTST/64                
Metrics                  MSE    RMSE     MAE
Country Pred_len                            
DE      24            0.0210  0.1450  0.0880
        96            0.0359  0.1894  0.1250
        168           0.0388  0.1968  0.1327
ES      24            0.0099  0.0993  0.0597
        96            0.0190  0.1377  0.0879
        168           0.0213  0.1460  0.0952
FR      24            0.0100  0.1002  0.0552
        96            0.0197  0.1405  0.0811
        168           0.0222  0.1490  0.0876
GB      24            0.0246  0.1570  0.0990
        96            0.0411  0.2027  0.1390
        168           0.0435  0.2085  0.1449
IT      24            0.0101  0.1005  0.0573
        96            0.0191  0.1380  0.0827
        168           0.0202  0.1421  0.0877